In [1]:
import io
import arcpy
import pandas as pd
import requests
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureSet
from arcgis.widgets import MapView
from zipfile import ZipFile
import os
from osgeo import ogr
from arcgis.geometry import SpatialReference
import sys


In [2]:
from arcgis import GIS
gis = GIS()

In [3]:
# PART 1, STEP 1:
# accessing lidar sample via api
dnr_link = r'https://resources.gisdata.mn.gov/pub/data/elevation/lidar/examples/lidar_sample/las/4342-12-05.las'

#download las file from MN DNR   
dnr_response = requests.get(dnr_link, allow_redirects=True)

In [4]:
#open a new file and use the dot interface. write to put data in the file
open("C:\\Users\\bende287\Downloads\\4342-12-05.las", 'wb').write(dnr_response.content)

389814899

In [5]:
arcpy.env.workspace = "C:\\Users\\bende287\Downloads"

#make Las dataset layer to transform into TIN and DEM
arcpy.management.MakeLasDatasetLayer('4342-12-05.las', 'las_layer')

<Result 'las_layer'>

In [6]:
#exports a triangulated irregular network from a LAS dataset- numbers for parameters were taken from documentation example!!
arcpy.ddd.LasDatasetToTin('las_layer', 'las_TIN', 'RANDOM', 15, 3.28)

<Result 'C:\\Users\\bende287\\Downloads\\las_TIN'>

In [7]:
#converting layer to DEM
arcpy.conversion.LasDatasetToRaster('las_layer', 'las.tif', 'ELEVATION')

<Result 'C:\\Users\\bende287\\Downloads\\las.tif'>

In [8]:

aprx = arcpy.mp.ArcGISProject("CURRENT")
aprx.defaultGeodatabase = r'C:\Users\bende287\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb'
aprx.save()
print(aprx.filePath)


C:\Users\bende287\Desktop\EsriTraining\PythEveryone\PythEveryone\PythonInArcGIS\Providence\GIS5571Lab1.aprx


In [9]:
#exporting TIN as PDF
m = aprx.listMaps("Map2")[0]
lytTIN = aprx.listLayouts("Layout TIN")[0]
lytTIN.exportToPDF(r"C:\Users\bende287\Documents\ArcGIS\Projects\MyProject1\Output\las_TIN1018.pdf", resolution=300)

'C:\\Users\\bende287\\Documents\\ArcGIS\\Projects\\MyProject1\\Output\\las_TIN1018.pdf'

In [10]:
#exporting DEM as PDF
lytDEM = aprx.listLayouts("Layout DEM")[0]
lytDEM.exportToPDF(r"C:\Users\bende287\Documents\ArcGIS\Projects\MyProject1\Output\las_DEM1018.pdf", resolution=300)

'C:\\Users\\bende287\\Documents\\ArcGIS\\Projects\\MyProject1\\Output\\las_DEM1018.pdf'

In [9]:
#STEP 2: do this in maps- no code needed

In [11]:
#STEP 3: 

#downloads the annual 30-Year Normals .bil files for precipitation from PRISM

prism_link = r'https://ftp.prism.oregonstate.edu/normals/monthly/4km/ppt/PRISM_ppt_30yr_normal_4kmM4_all_bil.zip'
prism_response = requests.get(prism_link, allow_redirects=True)

In [12]:
#open a new file and use the dot interface. write to put data in the file
open("C:\\Users\\bende287\Downloads\\PRISM_ppt_30yr_normal_4kmM4_all_bil.zip", 'wb').write(prism_response.content)


27839341

In [13]:
#unzip file
with ZipFile("C:\\Users\\bende287\Downloads\\PRISM_ppt_30yr_normal_4kmM4_all_bil.zip", 'r') as zip:
    zip.printdir()
    
    print('Extracting all files now..')
    zip.extractall("C:\\Users\\bende287\Downloads\\PRISM_ppt_30yr_normal_4kmM4_all_bil")
    print('Done!')

File Name                                             Modified             Size
PRISM_ppt_30yr_normal_4kmM4_01_bil.bil         2022-12-01 13:48:26      3490020
PRISM_ppt_30yr_normal_4kmM4_01_bil.bil.aux.xml 2022-12-05 12:01:10          386
PRISM_ppt_30yr_normal_4kmM4_01_bil.hdr         2022-12-01 13:48:26          316
PRISM_ppt_30yr_normal_4kmM4_01_bil.info.txt    2022-12-01 16:32:10          510
PRISM_ppt_30yr_normal_4kmM4_01_bil.prj         2022-12-01 13:48:26          166
PRISM_ppt_30yr_normal_4kmM4_01_bil.stx         2022-12-01 13:48:26           35
PRISM_ppt_30yr_normal_4kmM4_01_bil.xml         2022-12-05 12:01:10         8758
PRISM_ppt_30yr_normal_4kmM4_02_bil.bil         2022-12-01 13:48:26      3490020
PRISM_ppt_30yr_normal_4kmM4_02_bil.bil.aux.xml 2022-12-05 12:01:10          386
PRISM_ppt_30yr_normal_4kmM4_02_bil.hdr         2022-12-01 13:48:26          316
PRISM_ppt_30yr_normal_4kmM4_02_bil.info.txt    2022-12-01 16:32:10          510
PRISM_ppt_30yr_normal_4kmM4_02_bil.prj  

In [14]:
arcpy.management.CreateMosaicDataset(r'C:\Users\bende287\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb', '30_year_mosaic', 4269)

ExecuteError: ERROR 160326: The table already exists.
Failed to execute (CreateMosaicDataset).


In [15]:
#here we have 13 rasters to add -- 12 months, plus the annual
arcpy.management.AddRastersToMosaicDataset('T30_year_mosaic', 'Raster Dataset', r'C:\Users\bende287\Downloads\PRISM_ppt_30yr_normal_4kmM4_all_bil')

<Result 'T30_year_mosaic'>

In [16]:
#calculate timestamp and variable fields-- followed blog post
arcpy.management.CalculateField(
    in_table=r"T30_year_mosaic\Footprint",
    field="Variable",
    expression='"30yr Normal"',
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'T30_year_mosaic\\Footprint'>

In [17]:
arcpy.management.AddField("T30_year_mosaic", "Timestamp5", "TEXT", "", "", "")


arcpy.management.CalculateField(
    in_table=r"T30_year_mosaic\Footprint",
    field="Timestamp",
    expression="datetime.datetime(2023, int(!Name![-6:-4]), 1) if !Name![-6:-4].isdigit() else None",
    expression_type="PYTHON3",
    code_block="import datetime",
    field_type="DATE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)


<Result 'T30_year_mosaic\\Footprint'>

In [18]:
#build multidimensional info
arcpy.md.BuildMultidimensionalInfo(
    in_mosaic_dataset="T30_year_mosaic",
    variable_field="Variable",
    dimension_fields="Timestamp # #",
    variable_desc_units=None,
    delete_multidimensional_info="NO_DELETE_MULTIDIMENSIONAL_INFO"
)

<Result 'T30_year_mosaic'>

In [19]:
#convert to MD raster layer
arcpy.md.MakeMultidimensionalRasterLayer(
    in_multidimensional_raster="T30_year_mosaic",
    out_multidimensional_raster_layer="T30_year_mosaic_MultidimLayer1",
    variables="'30yr Normal'",
    dimension_def="ALL",
    dimension_ranges=None,
    dimension_values=None,
    dimension="",
    start_of_first_iteration="",
    end_of_first_iteration="",
    iteration_step=None,
    iteration_unit="",
    template='-125.020833333 24.0625000000001 -66.4791666663333 49.9375000000001 GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    dimensionless="DIMENSIONS",
    spatial_reference=None
)

<Result 'T30_year_mosaic_MultidimLayer1'>

In [20]:
#creates space time cube from multidimensional raster layer
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer(
    in_md_raster="T30_year_mosaic_MultidimLayer",
    output_cube=r"C:\Users\bende287\Documents\ArcGIS\Projects\MyProject1\spacetimecube1017.nc",
    fill_empty_bins="ZEROS"
)

<Result 'C:\\Users\\bende287\\Documents\\ArcGIS\\Projects\\MyProject1\\spacetimecube1017.nc'>

In [8]:
arcpy.stpm.VisualizeSpaceTimeCube3D(
    in_cube=r"C:\Users\bende287\Documents\ArcGIS\Projects\MyProject1\spacetimecube1017.nc",
    cube_variable="30YR_NORMAL_NONE_ZEROS",
    display_theme="VALUE",
    output_features=r"C:\Users\bende287\Documents\ArcGIS\Projects\MyProject1\MyProject1.gdb\spacetimecube1017_VisualizeSpaceTimeCube3D"
)

<Result 'C:\\Users\\bende287\\Documents\\ArcGIS\\Projects\\MyProject1\\MyProject1.gdb\\spacetimecube1017_VisualizeSpaceTimeCube3D'>